In [1]:
#Simple CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

class SimpleCNN:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize model with input shape
        model = Sequential()
        input_shape = (height, width, depth)
        chan_dim = -1
        
        # first CONV=>RELU=>BN layer set
        model.add(Conv2D(32, (3,3), strides=(2,2), padding='same', input_shape=input_shape))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chan_dim))
        
        # Second CONV=>RELU=>BN layer set
        model.add(Conv2D(64, (3,3), strides=(2,2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chan_dim))
        
        # First and only set of FC=>RELU layers
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation('softmax'))
        
        return model

In [2]:
# Generate Adversarial image
from tensorflow.keras.losses import MSE
import tensorflow as tf

def generate_image_adversary(model, image, label, eps=2/255.0):
    # cast the image
    image = tf.cast(image, tf.float32)
    
    # record gradients
    with tf.GradientTape() as tape:
        # explicitly indicate that our image should be tracked for growing gradients
        tape.watch(image)
        
        # use model to make predictions on input image and compute loss
        pred = model(image)
        loss = MSE(label, pred)
        
    # calculate gradients of loss w.r.t image then compute sign of gradient
    gradient = tape.gradient(loss, image)
    signed_grad = tf.sign(gradient)
    
    # construct the image adversary
    adversary = (image + (signed_grad * eps)).numpy()
    
    return adversary

In [3]:
# generating adversary images batch
import numpy as np

def generate_adversarial_batch(model, total, images, labels, dims, eps=0.01):
    # unpack image dimensions into convienence variables
    (h, w, c) = dims
    
    # since we are constructing a data generator here so we need to loop indefinitely
    while True:
        
        # initialize our preturbed images and labels
        preturbed_images = []
        preturbed_labels = []
        
        # randomly sample indexes (without replacement) from input data
        idx = np.random.choice(range(0, len(images)), size=total, replace=False)
            
        # loop over the indexes
        for i in idx:
                
            # grab current image and label
            image = images[i]
            label = labels[i]
                
            # generate an adversarial image
            adversary = generate_image_adversary(model, image.reshape(1, h, w, c),
                                                    label, eps)
                
            # update preturbed images and label lists
            preturbed_images.append(adversary.reshape(h, w, c))
            preturbed_labels.append(label)
                
        #yield preturbed images and labels
        yield (np.array(preturbed_images), np.array(preturbed_labels))

In [4]:
# training
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import numpy as np
import cv2 as cv

# load mnist dataset and scale the pixel values to range [0, 1]
print('[INFO] loading MNIST dataset...')
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0

# add a chennel dimension to images
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)

# one hot encode labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# initialize optimizer and model
print('[INFO] Compiling model...')
opt = Adam(lr=1e-3)
model = SimpleCNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# train simple CNN on MNIST
print('[INFO] Training Model...')
model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, 
          epochs=20, verbose=1)

# make predictions on testing set for model trained on non-adversarial images
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
print(f'[INFO] loss: {loss:.4f}, accuracy: {acc:.4f}')

[INFO] loading MNIST dataset...
11501568/11490434 [==============================] - 0s 0us/step
[INFO] Compiling model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[INFO] Training Model...
Epoch 1/20
938/938 [==============================] - 21s 6ms/step - loss: 0.2019 - accuracy: 0.9397 - val_loss: 0.0655 - val_accuracy: 0.9793
Epoch 2/20
938/938 [==============================] - 5s 6ms/step - loss: 0.0760 - accuracy: 0.9771 - val_loss: 0.0458 - val_accuracy: 0.9851
Epoch 3/20
938/938 [==============================] - 6s 6ms/step - loss: 0.0571 - accuracy: 0.9825 - val_loss: 0.0441 - val_accuracy: 0.9850
Epoch 4/20
938/938 [==============================] - 5s 6ms/step - loss: 0.0456 - accuracy: 0.9859 - val_loss: 0.0345 - val_accuracy: 0.9878
Epoch 5/20
938/938 [==============================] - 5s 6ms/step - loss: 0.0393 - accuracy: 0.9879 - val_loss: 0.0376 - val_accuracy: 0.9874
Epoch 6/20
938/938 [==============================] - 5s 6ms/step - loss: 0.0349 - accuracy: 0.9889 - val_loss: 0.0482 - val_accuracy: 0.9848
Epoch 7/20
938/938 [==============================] - 5s 6ms/step - loss: 0.0281 - accuracy: 0.9909 - val_loss: 0.0348 - v

In [5]:
# Testing on adversarial images

# generate a set of adversarial from our test set
print('[INFO] generating adversarial from our test set')
(advX, advY) = next(generate_adversarial_batch(model, len(testX), testX, testY, 
                                               (28, 28, 1), eps=0.1))

# re-evaluate model on adversarial images
(loss, acc) = model.evaluate(x=advX, y=advY, verbose=0)
print('[INFO] adversarial testing images:')
print(f'[INFO] loss: {loss:.4f}, accuracy: {acc:.4f}')

# since accuracy has dropped on the adversarial images we will fine-tune our model 
#on the adversarial images
# lower learning rate and re-compile model so we can fine tune it on adversairal images
print('[INFO] re-compiling model...')
opt = Adam(lr = 1e-4)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# fine tuning model on adversarial images
print('[INFO] fine-tuning network on adversarial images...')
model.fit(advX, advY, batch_size=64, epochs=15, verbose=1)


# now that our model is fine tuned we should evaluate it on test set (non-adversarial)
# to check if performance has degraded
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
print('')
print('[INFO] testing on normal images after fine-tuning:')
print(f'[INFO] loss: {loss:.4f}, accuracy: {acc:.4f}')

# final evaluation of model on adversarial images
(loss, acc) = model.evaluate(x=advX, y=advY, verbose=0)
print('[INFO] testing on adversarial images after fine-tuning:')
print(f'[INFO] loss: {loss:.4f}, accuracy: {acc:.4f}')

[INFO] generating adversarial from our test set
[INFO] adversarial testing images:
[INFO] loss: 13.0508, accuracy: 0.0166
[INFO] re-compiling model...
[INFO] fine-tuning network on adversarial images...
Epoch 1/15


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


157/157 [==============================] - 1s 5ms/step - loss: 8.6952 - accuracy: 0.2295
Epoch 2/15
157/157 [==============================] - 1s 5ms/step - loss: 2.9628 - accuracy: 0.6119
Epoch 3/15
157/157 [==============================] - 1s 5ms/step - loss: 1.3841 - accuracy: 0.7761
Epoch 4/15
157/157 [==============================] - 1s 5ms/step - loss: 0.8605 - accuracy: 0.8509
Epoch 5/15
157/157 [==============================] - 1s 5ms/step - loss: 0.5742 - accuracy: 0.8919
Epoch 6/15
157/157 [==============================] - 1s 5ms/step - loss: 0.3980 - accuracy: 0.9218
Epoch 7/15
157/157 [==============================] - 1s 5ms/step - loss: 0.3077 - accuracy: 0.9381
Epoch 8/15
157/157 [==============================] - 1s 5ms/step - loss: 0.2365 - accuracy: 0.9508
Epoch 9/15
157/157 [==============================] - 1s 5ms/step - loss: 0.1817 - accuracy: 0.9590
Epoch 10/15
157/157 [==============================] - 1s 5ms/step - loss: 0.1412 - accuracy: 0.9659
Epoch 11/1